In [ ]:
!pip install mediapipe

In [1]:
import numpy as np
import cv2 as cv
import os
import numpy as np
from glob import glob

In [2]:
input_path = os.path.join("dataset/raw_data") 
# output_path = os.path.join("dataset/preprocess_data")

# Actions that we try to detect
words = np.array(['hello', 'like', 'dislike'])

## Resize and flip videos and save

In [4]:
words = np.array(['hello', 'like', 'dislike'])
input_path = os.path.join("dataset/raw_data") 
output_path = os.path.join("dataset/preprocess_data")

for word in words:  
    a = 0
    for video_file in glob(os.path.join(input_path, word, "*")):
        
        result_flip = cv.VideoWriter(f'{output_path}/{word}/flip_{a}.mp4', 
                         cv.VideoWriter_fourcc(*'MP4V'),
                         10, (400,600))
        result = cv.VideoWriter(f'{output_path}/{word}/resize_{a}.mp4', 
                         cv.VideoWriter_fourcc(*'MP4V'),
                         10, (400,600))
        
        cap = cv.VideoCapture(video_file)
        
        while cap.isOpened():
            
            ret, image = cap.read()
            if ret == True:
                
                img_resize = cv.resize(image, (640, 480))
                img_flip = cv.flip(img_resize,1)
                
                result.write(img_resize)
                result_flip.write(img_flip)
#                 cv.imshow('frame',resize)
#                 if cv.waitKey(1) & 0xFF == ord('q'):
#                     break
            else:
                break
        a += 1

# cap.release()
# cv.destroyAllWindows()

In [15]:
cap.release()
cv.destroyAllWindows()

In [9]:
!pip install tqdm

     ---------------------------------------- 78.5/78.5 kB 1.1 MB/s eta 0:00:00


## Extract Features

In [1]:
import cv2 as cv
import numpy as np
import os
import mediapipe as mp
from glob import glob
import tqdm

In [2]:
mp_drawing_styles = mp.solutions.drawing_styles
mp_drawing = mp.solutions.drawing_utils # Drawing utilities
mp_hands = mp.solutions.hands

In [3]:
def mediapipe_detection(image, model):
    image = cv.cvtColor(image, cv.COLOR_BGR2RGB) # COLOR CONVERSION BGR 2 RGB
    image.flags.writeable = False                  # Image is no longer writeable
    results = model.process(image)                 # Make prediction
    image.flags.writeable = True                   # Image is now writeable 
    image = cv.cvtColor(image, cv.COLOR_RGB2BGR) # COLOR COVERSION RGB 2 BGR
    return image, results

In [4]:
def draw_landmarks(image, results):
    for hand_landmarks in results.multi_hand_landmarks:
        mp_drawing.draw_landmarks(image, hand_landmarks, mp_hands.HAND_CONNECTIONS,
        mp_drawing_styles.get_default_hand_landmarks_style(), mp_drawing_styles.get_default_hand_connections_style())

In [5]:
def extract_keypoints(results):
    keypoints = []
    if results.multi_hand_landmarks:
        for h_lmk in results.multi_hand_landmarks[0].landmark:
            keypoints.append(np.array([h_lmk.x, h_lmk.y, h_lmk.z]))
    else:
        keypoints.append(np.zeros(21*3))
    
    keypoints = np.array(keypoints).flatten()
    return keypoints


In [6]:
data_path = 'dataset/custom_raw_data/bro_alex_401_450/'
output_path = 'dataset/custom_numpy/'

with mp_hands.Hands( static_image_mode=True, max_num_hands=1, min_detection_confidence=0.5) as hands:
    for video_folder in os.listdir(data_path):
        for video_file in os.listdir(f'{data_path}/{video_folder}'):
            cap = cv.VideoCapture(f'{data_path}/{video_folder}/{video_file}')
            video_file_name = video_file.split('.')[0]
            if not os.path.exists(f'{output_path}/{video_folder}/{video_file_name}'):
                os.makedirs(os.path.join(f'{output_path}/{video_folder}/{video_file_name}'))

            num = 1
            while cap.isOpened():

                ret, frame = cap.read()
                if ret is False:
                    break

                frame = cv.resize(frame, (640,480))
                img, results = mediapipe_detection(frame, hands)

                if not results.multi_hand_landmarks:
                    continue

                draw_landmarks(img, results)

#                 cv.imshow('Detect landmarks', img)

                keypoints = extract_keypoints(results)         
                npy_path = os.path.join(f'{output_path}/{video_folder}/{video_file_name}/{num}')
                np.save(npy_path, keypoints)
            
                num+=1
                                   
#                 if cv.waitKey(1) & 0xFF == ord('q'):
#                     break
# #     print(keypoints)
#     cap.release()
#     cv.destroyAllWindows()

In [11]:
cap.release()
cv.destroyAllWindows()